In [249]:
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns

In [250]:
bls_work_stoppages_file_location = 'Dataset/work_stoppages_BLS.csv'  

def read_numeric_cell_with_commas(cell):
    try:
        return int(cell.replace(',',''))
    except:
        return 0

split_string_by_comma = lambda s: pd.Series(str(s).split(','))

dtypes = {
    'Organizations involved': 'category',
    'Ownership': 'category',
    'Industry code[1]': 'category'
}

renamed_columns = {
    'Days idle, cumulative for this work stoppage[3]': 'Cumulative days idle',
    'Number of workers[2]': 'Number of workers',
    'Work stoppage beginning date': 'Beginning date',
    'Work stoppage ending date': 'Ending date',
    'Industry code[1]': 'Industry code'

}

column_converters = {
    'States': split_string_by_comma,
    'Days idle, cumulative for this work stoppage[3]': read_numeric_cell_with_commas, 
    'Number of workers[2]': read_numeric_cell_with_commas
}

other_columns_to_use = ['States', 'Union acronym']


bls_dataframe = pd.read_csv(bls_work_stoppages_file_location, 
    dtype=dtypes, 
    usecols=list(dtypes) + list(renamed_columns) + other_columns_to_use, 
    parse_dates=['Work stoppage beginning date','Work stoppage ending date'],
    converters=column_converters)

bls_dataframe = bls_dataframe.rename(columns=renamed_columns)

bls_dataframe = bls_dataframe.explode(column='States')
bls_dataframe.head()



,Organizations involved,States,Ownership,Industry code,Union acronym,Beginning date,Ending date,Number of workers,Cumulative days idle
0,Marine Towing and Transportation Employers' As...,NY,Private industry,488330,ILA,1988-02-16,1993-12-20,2500,2879500
1,Boeing Company,WA,Private industry,336411,SPEEA,1993-01-19,1993-01-19,21000,21000
2,Boston Gas Company,MA,Private industry,22121,USW,1993-01-24,1993-05-20,1000,83000
3,Bituminous Coal Operators Association,IN,Private industry,212112,UMWA,1993-02-02,1993-03-02,6700,103400
3,Bituminous Coal Operators Association,IL,Private industry,212112,UMWA,1993-02-02,1993-03-02,6700,103400


In [251]:
bls_industry_codes_file_location = 'Dataset/bls_naics_2022_titles_descriptions.csv'

industry_codes_df = pd.read_csv(bls_industry_codes_file_location, usecols=['2022 NAICS','2022 NAICS Short Title'])

industry_codes_df = industry_codes_df.rename(columns={'2022 NAICS': 'Industry code', '2022 NAICS Short Title': 'Industry Name'})
industry_codes_df = industry_codes_df.set_index('Industry code')

bls_dataframe = bls_dataframe.join(industry_codes_df, on='Industry code').rename(columns={'Industry Name': 'Industry'})
bls_dataframe.head()

,Organizations involved,States,Ownership,Industry code,Union acronym,Beginning date,Ending date,Number of workers,Cumulative days idle,Industry
0,Marine Towing and Transportation Employers' As...,NY,Private industry,488330,ILA,1988-02-16,1993-12-20,2500,2879500,Navigational Services to Shipping
1,Boeing Company,WA,Private industry,336411,SPEEA,1993-01-19,1993-01-19,21000,21000,Aircraft Manufacturing
2,Boston Gas Company,MA,Private industry,22121,USW,1993-01-24,1993-05-20,1000,83000,Natural Gas Distribution
3,Bituminous Coal Operators Association,IN,Private industry,212112,UMWA,1993-02-02,1993-03-02,6700,103400,NaN
3,Bituminous Coal Operators Association,IL,Private industry,212112,UMWA,1993-02-02,1993-03-02,6700,103400,NaN


In [252]:
grouped_by_state = bls_dataframe.groupby(['States']).agg({
    'Number of workers': ['sum', 'mean', 'count'],
    'Cumulative days idle': ['sum','mean', 'count']
})

grouped_by_state.head(10)

Number of workers                     Cumulative days idle  \
                     sum          mean count                  sum   
States                                                              
                   36500  36500.000000     1              1204500   
 AL                 2700   1350.000000     2                90800   
 AZ                 4300   4300.000000     1                12900   
 CA               164800  23542.857143     7             17684200   
 CO               168100  33620.000000     5             21429500   
 CT                45200  11300.000000     4               116700   
 DC               241200  60300.000000     4             18366200   
 DE                89300  29766.666667     3               982500   
 FL               169600  24228.571429     7             17562200   
 GA               176700  22087.500000     8             17775400   

                            
                mean count  
States                      
        1.204500e+06     1  
 AL     4.540000e+04     2  
 AZ     1.290000e+04     1  
 CA     2.526314e+06     7  
 CO     4.285900e+06     5  
 CT     2.917500e+04     4  
 DC     4.591550e+06     4  
 DE     3.275000e+05     3  
 FL     2.508886e+06     7  
 GA     2.221925e+06     8

In [253]:
grouped_by_ownership = bls_dataframe.groupby(['Ownership']).agg({
    'Number of workers': ['sum', 'mean', 'count'],
    'Cumulative days idle': ['sum','mean', 'count']
})

grouped_by_ownership.head()

C:\Users\emmaf\AppData\Local\Temp\ipykernel_10452\3561215267.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_by_ownership = bls_dataframe.groupby(['Ownership']).agg({


Number of workers                      \
                                         sum          mean count   
Ownership                                                          
Local government                      637200   5845.871560   109   
Private industry                     8564400  12394.211288   691   
State and local government             15800  15800.000000     1   
State government                      851000  17020.000000    50   

                           Cumulative days idle                       
                                            sum           mean count  
Ownership                                                             
Local government                        3512200   32222.018349   109  
Private industry                      526363000  761740.955137   691  
State and local government               136500  136500.000000     1  
State government                        3031100   60622.000000    50

In [254]:
grouped_by_industry = bls_dataframe.groupby(['Industry']).agg({
    'Number of workers': ['sum', 'mean', 'count'],
    'Cumulative days idle': ['sum','mean', 'count']
})

grouped_by_industry.head()

Number of workers  \
                                                                 sum   
Industry                                                               
Administration of Human Resource Programs                       4500   
Administration of Human Resource Programs (exce...              2200   
Advertising Agencies                                         3105000   
Aerospace Product and Parts Manufacturing                       2300   
Agriculture, Construction, and Mining Machinery...             67800   

                                                                         \
                                                             mean count   
Industry                                                                  
Administration of Human Resource Programs             4500.000000     1   
Administration of Human Resource Programs (exce...    2200.000000     1   
Advertising Agencies                                135000.000000    23   
Aerospace Product and Parts Manufacturing             2300.000000     1   
Agriculture, Construction, and Mining Machinery...    9685.714286     7   

                                                   Cumulative days idle  \
                                                                    sum   
Industry                                                                  
Administration of Human Resource Programs                          4500   
Administration of Human Resource Programs (exce...                 6600   
Advertising Agencies                                          397440000   
Aerospace Product and Parts Manufacturing                         29900   
Agriculture, Construction, and Mining Machinery...             16295300   

                                                                      
                                                          mean count  
Industry                                                              
Administration of Human Resource Programs               4500.0     1  
Administration of Human Resource Programs (exce...      6600.0     1  
Advertising Agencies                                17280000.0    23  
Aerospace Product and Parts Manufacturing              29900.0     1  
Agriculture, Construction, and Mining Machinery...   2327900.0     7